In [1]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# %matplotlib inline
# import seaborn as sns
import json
import re
import spacy
nlp = spacy.load('en')

Common Preprocessing Steps
http://pages.cs.wisc.edu/~jerryzhu/cs769/text_preprocessing.pdf
Preprocessing text is called tokenization or text normalization
1. Throw away unwanted stuff(e.g., HTML tags)
2. Word boundaries: white space and punctuations
3. Stemming (Lemmatization
4. Stopword removal
5. Capitalization, case folding

https://datascience.stackexchange.com/questions/11402/preprocessing-text-before-use-rnn/11421

ML Workflow: https://www.pewe.sk/datalys/wp-content/uploads/sites/3/2017/10/2017-10-05-Datalys-ML-Workflow.pdf

https://www.kdnuggets.com/tag/data-preprocessing
https://www.kdnuggets.com/2017/12/general-approach-preprocessing-text-data.html
https://www.kdnuggets.com/2018/05/getting-started-spacy-natural-language-processing.html

Preprocess review comments for word2vac model
preprocess for sentiment analysis

create bigrams
https://stackoverflow.com/questions/46129335/get-bigrams-and-trigrams-in-word2vec-gensim

In [2]:
all_adj = []       
def get_adj(comment):
    temp_adjs = dict()
    for token in nlp(comment):
        if token.pos_ == 'ADJ':
            word = token.lemma_
            temp_adjs[word] = temp_adjs.get(word,0)+1
     
    return temp_adjs

# for key, value in d1.items():
#         d2[key] = d2.get(key,0) + value

In [3]:
reviews = pd.read_json('/home/zafrin/PycharmProjects/RateMyProfessor/data/raw/reviews_old_play.json')

In [63]:
reviews.head(2)

,attendence,class_names,comments,date,for_credits,grades_revieved,level_of_difficulty,overall_quality,prof_id,rating_type,tags_by_user,textbook_used,thumbs_down,thumbs_up,would_take_again
0,Mandatory,SOC106,Clawson is really caring and interested in his...,2017-12-14,Yes,N/A,3,4.0,100001,good,"[CARING, RESPECTED, GET READY TO READ]",No,0,0,Yes
1,Mandatory,SOC353,This class was definitely not what I thought i...,2017-11-15,N/A,B,3,3.0,100001,average,"[CLEAR GRADING CRITERIA, PARTICIPATION MATTERS...",Yes,0,1,No


In [ ]:
reviews['adjs'] = reviews.comments.copy()
reviews['adjs'] = reviews.adjs.apply(get_adj)

In [1]:
reviews.adjs

NameError: name 'reviews' is not defined

In [14]:
reviews_adjs = reviews[['prof_id', 'adjs']]

In [15]:
reviews_adjs.head()

,prof_id,adjs
0,100001,"{'interested': 1, '-PRON-': 3, 'easy': 2, 'whi..."
1,100001,"{'fine': 1, 'big': 1, '-PRON-': 1, 'okay': 1, ..."
2,100001,{}
3,100001,"{'good': 1, 'interested': 1, 'awesome': 1}"
4,100001,"{'wonderful': 1, 'great': 1, 'good': 1}"


In [ ]:
adjs_df = reviews_adjs['adjs'].apply(pd.Series)

In [ ]:
reviews_adjs_final = pd.concat([reviews_adjs,adjs_df], axis=1).drop('adjs', axis=1)

In [ ]:
groupby

In [ ]:
import pickle
pickle_out = open('/home/zafrin/PycharmProjects/RateMyProfessor/data/pickle/review_adj.pickle', 'wb')
pickle.dump(reviews_adjs, pickle_out)
pickle_out.close()

In [64]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183186 entries, 0 to 183185
Data columns (total 15 columns):
attendence             183186 non-null object
class_names            183186 non-null object
comments               183186 non-null object
date                   183186 non-null datetime64[ns]
for_credits            183186 non-null object
grades_revieved        183186 non-null object
level_of_difficulty    183186 non-null int64
overall_quality        183186 non-null float64
prof_id                183186 non-null int64
rating_type            183186 non-null object
tags_by_user           183186 non-null object
textbook_used          183186 non-null object
thumbs_down            183186 non-null int64
thumbs_up              183186 non-null int64
would_take_again       183186 non-null object
dtypes: datetime64[ns](1), float64(1), int64(4), object(9)
memory usage: 22.4+ MB


In [4]:
sample_review = reviews.comments[10]
sample_review

"horrible class. DONT take it.  dan doesn't talk about anything real during class, just rambles on about his opinion on stuff and then will give you random facts to write down so you can write the essays well. paper due every week, do all of them. and id write all the essays so that you can get a good grade. go to class because of random quizzes."

In [5]:
%%time
parsed_review = nlp(sample_review)

CPU times: user 49.1 ms, sys: 3.83 ms, total: 53 ms
Wall time: 93.5 ms


In [6]:
parsed_review

horrible class. DONT take it.  dan doesn't talk about anything real during class, just rambles on about his opinion on stuff and then will give you random facts to write down so you can write the essays well. paper due every week, do all of them. and id write all the essays so that you can get a good grade. go to class because of random quizzes.

In [7]:
for num, sent in enumerate(parsed_review.sents):
    print('Sentence {}:\n{}\n'.format(num+1, sent))

Sentence 1:
horrible class.

Sentence 2:
DONT take it.  

Sentence 3:
dan doesn't talk about anything real during class, just rambles on about his opinion on stuff and then will give you random facts to write down so you can write the essays well.

Sentence 4:
paper due every week, do all of them.

Sentence 5:
and id write all the essays so that you can get a good grade.

Sentence 6:
go to class because of random quizzes.



In [27]:
for num, entity in enumerate(parsed_review.ents):
    print('Entity {}:'.format(num + 1), entity, '-', entity.label_)

Entity 1: every week - DATE


In [10]:
df = pd.DataFrame(columns=['text', 'lemma', 'pos', 'tag', 'dep', 'shape', 'alpha', 'stop'])
for token in parsed_review:
    token_info = [token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
          token.shape_, token.is_alpha, token.is_stop]
    df2 = pd.DataFrame([token_info], columns=['text', 'lemma', 'pos', 'tag', 'dep', 'shape', 'alpha', 'stop'])
    #df = pd.DataFrame(np.insert(df.values, 0, values=[token_info], axis=0))
    df = df.append(df2)
df

,text,lemma,pos,tag,dep,shape,alpha,stop
0,horrible,horrible,ADJ,JJ,amod,xxxx,True,False
0,class,class,NOUN,NN,ROOT,xxxx,True,False
0,.,.,PUNCT,.,punct,.,False,False
0,DONT,dont,NOUN,NNS,nsubj,XXXX,True,False
0,take,take,VERB,VBP,ROOT,xxxx,True,True
0,it,-PRON-,PRON,PRP,dobj,xx,True,True
0,.,.,PUNCT,.,punct,.,False,False
0,,,SPACE,,,,False,False
0,dan,dan,NOUN,NN,nsubj,xxx,True,False
0,does,do,VERB,VBZ,aux,xxxx,True,True


In [71]:
reviews[reviews['grades_revieved'] == ' A']

,attendence,class_names,comments,date,for_credits,grades_revieved,level_of_difficulty,overall_quality,prof_id,rating_type,tags_by_user,textbook_used,thumbs_down,thumbs_up,would_take_again
34,Mandatory,SOC106,he likes to get in ur face,2005-11-30,N/A,A,2,4.0,100001,good,[],N/A,0,0,N/A
166,Mandatory,FINITEMATH,"IM TERRIBLE AT MATH TO BEGIN WITH, BUT GIRARDI...",2005-12-02,N/A,A,4,4.0,100013,good,[],N/A,0,0,N/A
232,Mandatory,SPAN301,I had taken two classes with Couture and will ...,2016-10-06,Yes,A,2,5.0,100017,awesome,"[RESPECTED, HILARIOUS, EXTRA CREDIT]",Yes,0,0,Yes
235,Mandatory,SPAN231232,An amazing professor who has definitely helped...,2014-12-10,Yes,A,2,4.5,100017,awesome,"[RESPECTED, CLEAR GRADING CRITERIA]",Yes,0,0,Yes
236,Mandatory,SPAN231,A fantastic Spanish teacher; he really knows h...,2014-12-02,N/A,A,4,4.0,100017,good,"[SKIP CLASS? YOU WON'T PASS., PARTICIPATION MA...",Yes,0,0,N/A
237,Mandatory,SPAN231,Awesome Professor! Very helpful and not much w...,2014-11-10,Yes,A,2,4.5,100017,awesome,[],Yes,0,0,N/A
316,Mandatory,HFT4277,She is a great professor. I took the class whi...,2016-11-30,Yes,A,2,5.0,100021,awesome,"[INSPIRATIONAL, ACCESSIBLE OUTSIDE CLASS]",Yes,0,0,Yes
318,N/A,HFT4277,"She has long and sometimes boring lectures, bu...",2016-04-23,Yes,A,1,3.5,100021,good,[LECTURE HEAVY],No,0,0,Yes
321,Mandatory,HFT4277,Such an easy and interesting class! Took her m...,2014-12-18,Yes,A,2,5.0,100021,awesome,"[SKIP CLASS? YOU WON'T PASS., PARTICIPATION MA...",Yes,0,0,N/A
432,Mandatory,DANC221,Deborah is one of the most amazing teachers I ...,2015-05-07,Yes,A,4,5.0,100036,awesome,"[RESPECTED, INSPIRATIONAL, GIVES GOOD FEEDBACK]",No,0,0,N/A


In [70]:
reviews['grades_revieved'].unique()

array([' N/A', ' B', ' A', ' Not sure yet', ' C', ' B+', ' A+', ' F',
       ' A-', ' D', ' B-', ' C+', ' WD', ' C-', ' P', ' INC', ' D+', ' D-',
       ' Audit/No Grade'], dtype=object)

In [1]:
import pyspark

ModuleNotFoundError: No module named 'pyspark'

In [2]:
import findspark

In [3]:
findspark.init('/home/zafrin/spark-2.3.1-bin-hadoop2.7')

In [4]:
import pyspark

In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [105]:
df = spark.read.json('/home/zafrin/PycharmProjects/RateMyProfessor/data/raw/reviews_old_play.json')

In [25]:
df.columns

['_corrupt_record',
 'attendence',
 'class_names',
 'comments',
 'date',
 'for_credits',
 'grades_revieved',
 'level_of_difficulty',
 'overall_quality',
 'prof_id',
 'rating_type',
 'tags_by_user',
 'textbook_used',
 'thumbs_down',
 'thumbs_up',
 'would_take_again']

In [40]:
df.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- attendence: string (nullable = true)
 |-- class_names: string (nullable = true)
 |-- comments: string (nullable = true)
 |-- date: string (nullable = true)
 |-- for_credits: string (nullable = true)
 |-- grades_revieved: string (nullable = true)
 |-- level_of_difficulty: string (nullable = true)
 |-- overall_quality: string (nullable = true)
 |-- prof_id: long (nullable = true)
 |-- rating_type: string (nullable = true)
 |-- tags_by_user: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- textbook_used: string (nullable = true)
 |-- thumbs_down: string (nullable = true)
 |-- thumbs_up: string (nullable = true)
 |-- would_take_again: string (nullable = true)



In [43]:
df.describe().show()

+-------+--------------------+--------------+--------------------+--------------------+----------+-----------+---------------+-------------------+------------------+------------------+-----------+-------------+-------------------+------------------+----------------+
|summary|     _corrupt_record|    attendence|         class_names|            comments|      date|for_credits|grades_revieved|level_of_difficulty|   overall_quality|           prof_id|rating_type|textbook_used|        thumbs_down|         thumbs_up|would_take_again|
+-------+--------------------+--------------+--------------------+--------------------+----------+-----------+---------------+-------------------+------------------+------------------+-----------+-------------+-------------------+------------------+----------------+
|  count|                   2|        183185|              183185|              183185|    183185|     183185|         183185|             183185|            183185|            183185|     183185|   

In [44]:
df['comments']

Column<b'comments'>

In [47]:
df.select('comments').show(5)

+--------------------+
|            comments|
+--------------------+
|                null|
|This class was de...|
|This class sucks....|
|Dan is best. I re...|
|Wonderful profess...|
+--------------------+
only showing top 5 rows



In [51]:
type(df['comments'])

pyspark.sql.column.Column

In [53]:
type(df.select('comments'))

pyspark.sql.dataframe.DataFrame

In [50]:
df.head(2)

[Row(_corrupt_record='[{"prof_id": 100001, "date": "12/14/2017", "rating_type": "good", "overall_quality": "4.0", "level_of_difficulty": "3.0", "class_names": "SOC106", "for_credits": " Yes", "attendence": " Mandatory", "textbook_used": " No", "would_take_again": " Yes", "grades_revieved": " N/A", "tags_by_user": ["CARING", "RESPECTED", "GET READY TO READ"], "comments": "Clawson is really caring and interested in his students so it\'s an easy class to pass, although there is A LOT of reading which is important to quizzes (a pretty big part of your grade) and the final exam. There\'s a big paper but its very doable and Dan is very easy to work with. Lectures get a bit tedious but can be though provoking", "thumbs_up": "0", "thumbs_down": "0"},', attendence=None, class_names=None, comments=None, date=None, for_credits=None, grades_revieved=None, level_of_difficulty=None, overall_quality=None, prof_id=None, rating_type=None, tags_by_user=None, textbook_used=None, thumbs_down=None, thumbs_

In [49]:
df.head(2)[0]

Row(_corrupt_record='[{"prof_id": 100001, "date": "12/14/2017", "rating_type": "good", "overall_quality": "4.0", "level_of_difficulty": "3.0", "class_names": "SOC106", "for_credits": " Yes", "attendence": " Mandatory", "textbook_used": " No", "would_take_again": " Yes", "grades_revieved": " N/A", "tags_by_user": ["CARING", "RESPECTED", "GET READY TO READ"], "comments": "Clawson is really caring and interested in his students so it\'s an easy class to pass, although there is A LOT of reading which is important to quizzes (a pretty big part of your grade) and the final exam. There\'s a big paper but its very doable and Dan is very easy to work with. Lectures get a bit tedious but can be though provoking", "thumbs_up": "0", "thumbs_down": "0"},', attendence=None, class_names=None, comments=None, date=None, for_credits=None, grades_revieved=None, level_of_difficulty=None, overall_quality=None, prof_id=None, rating_type=None, tags_by_user=None, textbook_used=None, thumbs_down=None, thumbs_u

In [73]:
df.filter(df['grades_revieved']==' A')

DataFrame[_corrupt_record: string, attendence: string, class_names: string, comments: string, date: string, for_credits: string, grades_revieved: string, level_of_difficulty: string, overall_quality: string, prof_id: bigint, rating_type: string, tags_by_user: array<string>, textbook_used: string, thumbs_down: string, thumbs_up: string, would_take_again: string]

In [74]:
c = df.filter(df['grades_revieved']==' A').collect()

In [75]:
c[0].asDict()

{'_corrupt_record': None,
 'attendence': ' Mandatory',
 'class_names': 'SOC106',
 'comments': 'he likes to get in ur face',
 'date': '11/30/2005',
 'for_credits': 'N/A ',
 'grades_revieved': ' A',
 'level_of_difficulty': '2.0',
 'overall_quality': '4.0',
 'prof_id': 100001,
 'rating_type': 'good',
 'tags_by_user': [''],
 'textbook_used': ' N/A',
 'thumbs_down': '0',
 'thumbs_up': '0',
 'would_take_again': ' N/A'}

In [80]:
df.groupby('prof_id').count().show(5)

+-------+-----+
|prof_id|count|
+-------+-----+
| 100565|   98|
| 100847|   42|
| 100852|   24|
| 101011|   81|
| 101155|   10|
+-------+-----+
only showing top 5 rows



In [82]:
df.agg({'prof_id': 'count'}).show()

+--------------+
|count(prof_id)|
+--------------+
|        183185|
+--------------+



In [85]:
gdf = df.groupby('prof_id')
gdf.agg({'prof_id': 'count'}).show(5)

+-------+--------------+
|prof_id|count(prof_id)|
+-------+--------------+
| 100565|            98|
| 100847|            42|
| 100852|            24|
| 101011|            81|
| 101155|            10|
+-------+--------------+
only showing top 5 rows



In [89]:
df.select('grades_revieved').distinct().show()

+---------------+
|grades_revieved|
+---------------+
|   Not sure yet|
|            INC|
|             B-|
|           null|
|             WD|
|             C-|
|              C|
|             B+|
|             D+|
|             D-|
|             A-|
| Audit/No Grade|
|              B|
|              A|
|              F|
|            N/A|
|              D|
|             C+|
|              P|
|             A+|
+---------------+



In [107]:
for i in df.select('grades_revieved').distinct().collect():
    print(i)

Row(grades_revieved=' Not sure yet')
Row(grades_revieved=' INC')
Row(grades_revieved=' B-')
Row(grades_revieved=None)
Row(grades_revieved=' WD')
Row(grades_revieved=' C-')
Row(grades_revieved=' C')
Row(grades_revieved=' B+')
Row(grades_revieved=' D+')
Row(grades_revieved=' D-')
Row(grades_revieved=' A-')
Row(grades_revieved=' Audit/No Grade')
Row(grades_revieved=' B')
Row(grades_revieved=' A')
Row(grades_revieved=' F')
Row(grades_revieved=' N/A')
Row(grades_revieved=' D')
Row(grades_revieved=' C+')
Row(grades_revieved=' P')
Row(grades_revieved=' A+')


In [98]:
from pyspark.sql.functions import *

In [121]:
df2 = df.withColumn('grade', ltrim(df['grades_revieved']))

In [122]:
for i in df2.select('grades_revieved').distinct().collect():
    print(i)

Row(grades_revieved=' Not sure yet')
Row(grades_revieved=' INC')
Row(grades_revieved=' B-')
Row(grades_revieved=None)
Row(grades_revieved=' WD')
Row(grades_revieved=' C-')
Row(grades_revieved=' C')
Row(grades_revieved=' B+')
Row(grades_revieved=' D+')
Row(grades_revieved=' D-')
Row(grades_revieved=' A-')
Row(grades_revieved=' Audit/No Grade')
Row(grades_revieved=' B')
Row(grades_revieved=' A')
Row(grades_revieved=' F')
Row(grades_revieved=' N/A')
Row(grades_revieved=' D')
Row(grades_revieved=' C+')
Row(grades_revieved=' P')
Row(grades_revieved=' A+')


In [33]:
corp = [[(0,3), (1,5)],
[(0,2), (2,3)],
[(2,1)],
[(0,1), (2,2)]]

In [37]:
vocabs = [0,1,2]
vcount = dict()
for bow in corp:
    for vocab, count in bow:
        vcount[vocab] = vcount.get(vocab, 0) +  count

In [38]:
vcount

{0: 6, 1: 5, 2: 6}

In [ ]:
get unique prof_id
make a temp df for that prof
in a new df insert {vcount}, review count, etc